# Part 2 - Understanding FITS files and Plotting Astronomical Images using AstroPy


In today's tutorial you will be learning about FITS files, which are a common filetype used to store astronomical data. Over the course of this tutorial, you will explore how to:

- download data stored in online archives
- open FITS files using Python packages and load image data
- explore the header information store in an FITS file
- edit a FITS header
- combine multiple FITS files to create a 3-color image

Before working through today's notebook we suggest you read the following chapters in the textbook:

 - Data I/O (Ch 5)
 - Plotting Astronomical Images (Ch 7)

## Section 1: Working with FITS files

###  Downloading data from an online archive

Let's begin by first accessing our data! For today's exercise we will be using data taken by the Las Cumbres Observatory (LCO) telescope network. The LCO archive is open to the public and can be accessed by going to https://archive.lco.global.

For this activity, we will be using data from a very specific set of observations of NGC 7635 or "The Bubble Nebula". The Bubble Nebula is an HII region emission nebula and was discovered in 1787 by William Herschel. It is located around 7,100 light-years away in the constellation Cassiopeia. Stellar wind from a central hot, massive star creates the “bubble”.

First, let's mount our Google Drive as we've done before to prep our Google Drive space to save the data:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Next, we need to make a directory in our Google Drive space to house our data. Let's call this folder "data":

In [ ]:
import os
os.mkdir("/content/drive/MyDrive/data/")

If you load up your Google Drive in a new browser tab, you should now see the directory there. 

Another way of checking is by clicking on the folder icon in the left-hand side bar, which shows the full directory tree of your Google Drive.

Now let's use `wget` a procedure for downloading files, to snag the data we will be working with:

In [ ]:
!wget -O /content/drive/MyDrive/data/LCO_NGC7635_B-band.fits https://www.dropbox.com/s/7d1gdkmcqr4wtaj/ogg0m406-kb27-20201107-0173-e91.fits?dl=0

In the cell below, write a `wget` command that would allow us to download a R filter file located at the following URL:

https://www.dropbox.com/s/uyqldfawhlmg1hw/ogg0m406-kb27-20201107-0171-e91.fits?dl=0

In [ ]:
#YOUR CODE HERE

Note: If you like you can also see all the data for this series of observations by heading over to this archive [link](https://archive.lco.global/?reduction_level=91&request_id=2286975&start=2014-08-01%2000%3A00&end=2023-01-31%2023%3A59&public=true&limit=20):

#### Let's start working with the data!

Here, we begin by loading in some important packages that we'll be using:

In [ ]:
from datetime import datetime
import numpy as np

# Set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.io import fits

### Opening FITS files and loading the image data

Let's open the FITS file to find out what it contains! Edit the cell below such that the variable "fits_1" is assigned the file name of one of the files you have downloaded today.

In [ ]:
fits_1 = '/content/drive/MyDrive/data/LCO_NGC7635_B-band.fits'

Now let's open up this file and get some info on what the file contains:

In [ ]:
hdu_list = fits.open(fits_1)

hdu_list.info()

A FITS file is comprised of segments called **Header Data Units** (HDUs), where the first HDU is called the "Primary HDU", or "Primary Array". An HDU is the highest level component of the FITS file structure, consisting of a header and (typically) a data array or table.

A `hdulist`, like you see in the output above, behaves like a Python list, and each element maps to a HDU in the FITS file.

***

#### Q - How many HDUs does this FITS file have? What do you think is the purpose for each one?

***

Ans.

Each HDU object has two important attributes: 

 - **data**, which behaves like a Numpy array, can be used to access the data
 - **header**, which behaves like a dictionary, can be used to access the header information which is information about the data itself 

`astropy.io.fits` provides a lot of flexibility for reading FITS files and headers, but most of the time the convenience functions are the easiest way to access the data. `fits.getdata()` reads only the data from a FITS file, but with the header=True keyword argument will also read the header.

Let's try this below:

In [ ]:
data, header = fits.getdata(fits_1, header=True)

There is also a dedicated function for reading only the header, called `fits.getheader()`:

In [ ]:
hdu_number = 0 # Primary HDU

fits.getheader(fits_1, hdu_number)

***

#### Q - Scroll through the header above. What types of information does this header contain? Why might headers be important for astronomers to include in their datafiles?

***

Ans.

Because `getdata()` can get both the data and the header, it's a very useful command to remember! Since the primary HDU of a FITS file must contain image data, the data is now stored in a numpy array. The header is stored in an object that acts like a standard Python dictionary.

We can access individual header keywords using standard item notation in Python. For example, if we are curious about where the data was acquired, we can use the `SITE` card.

In [ ]:
header["SITE"]

This tells us this image was recorded at Haleakala Observatory.

We can also take a look at the data:

In [ ]:
data.shape

This tells us that the data is stored in a 2-dimensional array that is **2042 x 3054 pixels** in size.

Modifying data or header information in a FITS file object is easy. We can update existing header keywords and even _create_ new keywords like as follows:

In [ ]:
now = datetime.now()

timenow = now.strftime("%H:%M:%S")

print(timenow)

header['MODIFIED'] = timenow  # adds a new keyword

In the cell below, write a statement that reads out the exposure time keyword. How long did the telescope integrate on this image?

In [ ]:
#Your Code Here

As we stated above, HDU objects have two parts: a header and an array of data.

Let's begin to explore the data component of the primary HDU by plotting it into a figure.

In [ ]:
plt.imshow(data[:,:], origin='lower')

As you can see, the image looks not so remarkable.. it almost looks featureless!

For one, it's a little small. We can increase its generated size using the `rcParams` attribute.

Also.. while we observe a few stars the image looks quite faded. Astronomers will often apply _image normalization_ here to improve the visualization of data. In image processing, a normalization is a process that changes the range of pixel intensity values. 

Let's try it!

In [ ]:
from astropy.visualization import (MinMaxInterval, SqrtStretch,
                                   ImageNormalize)

# Create an ImageNormalize object
norm = ImageNormalize(data[:,:], interval=MinMaxInterval(),
                      stretch=SqrtStretch())

plt.rcParams['figure.figsize'] = [12, 8]
plt.imshow(data[:,:], origin='lower', vmin=-0.0, vmax=15356.9, norm=norm)

We can now observe many more features in the data!

Note that this is just a plot of an array, so the coordinates are just pixel coordinates at this stage. The data is stored with _longitude_ increasing to the _right_ (the opposite of the normal convention in astronomy).

Now let's save our changes by writing to a *new* fits file using the `writeto` process in AstroPy:

In [ ]:
fits.writeto('my_modified_file.fits', data, header, overwrite=True)

Now let's load in the new file and readout the header. You should see a new card created that has a value assigned with the current time and date!

In [ ]:
fits.getheader('my_modified_file.fits', hdu_number)

We can use AstroPy to also get some statistics about our image:

In [ ]:
print('Min:', np.min(data))
print('Max:', np.max(data))
print('Mean:', np.mean(data))
print('Stdev:', np.std(data))

***

#### Q - What do you think these statistics reference? For example, the cell above outputs a min value and a max value. What do these min and max values represent?

***

Ans.

### Plotting out the Data

We can also use Python to visualize our astronomical images in different ways. For example, we notice in the above image a number of stars. We can slice our image in such a way that we select out only a small region of the above image and plot that out.



select out a row or column of data that passes through a star and plot out a histogram of this data sub-set.

for features in the image by collapsing it along one axis and plotting out a histogram. to do this we'll be first using `ndarray.flatten()` to collapse the data and returna  1D numpy array. Then, to make a histogram we will use `matplotlib.pyplot.hist()`.

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.imshow(data[1521:1671,2013:2163], origin='lower', vmin=0.0, vmax=15356.9, norm=norm)

We can also slice out a single row of data which passes through the center of the star and plot this out in 1D:

In [ ]:
plt.plot(data[1596, 2013:2163], label="pixel values")
plt.legend(fontsize="x-large")

Can you write a block of code that produces a similar line plot passing through one of the fainter stars in the image? Write this code block below.

In [ ]:
#Your Code Here

## World Coordinate Systems in FITS files

Also stored within the header of a FITS file is information regarding the image's **world coordinate system** or **WCS**. World Coordinate Systems describe the geometric transformations between one set of coordinates and another. A common application (as seen in this dataset) is to map the pixels in an image onto the celestial sphere. Another common application is to map pixels to wavelength in a spectrum.

`astropy.wcs` contains utilities for managing World Coordinate System (WCS) transformations defined in several elaborate FITS WCS standard conventions. These transformations work both forward (from pixel to world) and backward (from world to pixel). Let's explore that a bit here. 

Let's begin by first loading in `astropy.wcs`:

In [ ]:
from astropy.wcs import WCS

Now let's load in the WCS information stored in the header of the FITS file:

In [ ]:
wcs = WCS(header)

We can then print out this information to get a better idea of the coordinate system used in our image:

In [ ]:
print(wcs)

It's important to unpack this information! Here's what these values mean:

 - **CTYPE**: The TYPE of coordinate system. Here, it is Right Ascension (R.A.) and Declination (Dec). You can learn more about R.A. and Dec as a coordinate system [here](https://skyandtelescope.org/astronomy-resources/right-ascension-declination-celestial-coordinates/).
 - **CRPIX**: The reference value pixel
 - **CRVAL**: The R.A. and Dec values for the reference pixel
 - **CD values**: These values represent the values that comprise a linear transformation matrix. This matrix tells the computer how the coordinate system axes are oriented in relation to the array axes.
 
 
 Together these values allow astronomer to make the transformation between array/pixel coordinates to astronomical coordinates. More information on WCS and related keywords can be found [here](https://fits.gsfc.nasa.gov/standard40/fits_standard40aa-le.pdf).
 
 So now let's load in the WCS to our image so that the vertical and horizontal axes are properly labeled.

In [ ]:
plt.clf()
ax = plt.subplot(projection=wcs)
plt.imshow(data, origin='lower', vmin=0.0, vmax=15356.9, norm=norm)
ax.set_xlabel('Right Ascension', fontsize=18)
ax.set_ylabel('Declination', fontsize=18)
plt.show()

We can also overlay grid lines in white which show the grid of the WCS:

In [ ]:
ax = plt.subplot(projection=wcs)
plt.imshow(data, origin='lower', vmin=0.0, vmax=22318.7, norm=norm)
plt.grid(color='white', ls='solid')
ax.set_xlabel('Right Ascension', fontsize=18)
ax.set_ylabel('Declination', fontsize=18)

Indeed, there is lots that can be done with `astropy.wcs` and `matplotlib` in terms of creating really complex and rich figures of astronomical data for publications. More information on editing and controlling a figures axes/ticks/labels can be found [here]("https://docs.astropy.org/en/stable/visualization/wcsaxes/ticks_labels_grid.html).

You can even try editing this notebook to incorporate some of the plotting techniques in that tutorial using this dataset!

## Making a 3-Color Images from Astronomical Data

Satellite and spacecraft images are not visual color photographs but pictorial presentations of measured data. Image collection systems measure electromagnetic radiation in different bands of the electromagnetic spectrum (i.e. radio, infrared, visible, ultraviolet, X-ray, gamma ray). The color images you often see are in most cases assembled from separate black & white images taken through special types of filters. To produce one color image, three separate images are collected through a red, a green, and a blue filter. The three images are then combined with image analysis software into one color image.

For this reason, astronomical images are not “true color” images – they do not represent visually accurate color – they use “false” colors or more accurately “representative” colors. True color would be an attempt to reproduce a visually accurate color. False color is an arbitrary selection of colors to represent some characteristic in the image, such as photon intensity, energy, temperature and chemical composition/distribution, or features such as shock waves and magnetic fields.

In this section we're going to be exploring how astronomers construct 3-color images using data from the Sloan Digital Sky Survey (SDSS).

I've hosted three FITS files at the following URLs:


*   _i_ filter: https://www.dropbox.com/s/h61jlyjsehrueb9/reprojected_sdss_i.fits?dl=0
*   _g_ filter: https://www.dropbox.com/s/atgzojgedcl8f7m/reprojected_sdss_g.fits?dl=0
*   _r_ filter: https://www.dropbox.com/s/fvw4w5tfpc5vr6h/reprojected_sdss_r.fits?dl=0

Write a series of `wget` commands that download these files into your `data` folder on Drive.



In [ ]:
# YOUR CODE HERE

In [ ]:
i_data, i_header = fits.getdata('/content/drive/MyDrive/data/reprojected_sdss_i.fits', header=True)
r_data, r_header = fits.getdata('/content/drive/MyDrive/data/reprojected_sdss_r.fits', header=True)
g_data, g_header = fits.getdata('/content/drive/MyDrive/data/reprojected_sdss_g.fits', header=True)

Write out a code block below which reads in the header of this file. What type of coordinate system does this FITS file use?

In [ ]:
#YOUR CODE HERE

Next we're going to load in the `make_lupton_rgb` convenience function from `astropy.visualization`.

In [ ]:
from astropy.visualization import make_lupton_rgb

[Lupton et al. (2004)](https://ui.adsabs.harvard.edu/abs/2004PASP..116..133L/abstract) describe an “optimal” algorithm for producing red-green- blue composite images from three separate high-dynamic range arrays. This method is implemented in `make_lupton_rgb` as a convenience wrapper function and an associated set of classes to provide alternate scalings.

The next step is to then use this function to produce a 3-color RBG image from the *i*, *r*, and *g* filters data and display it using `matplotlib`.

In [ ]:
rgb_default = make_lupton_rgb(i_data, r_data, g_data, filename="ngc6976-default.jpeg")
ax2 = plt.subplot(projection=wcs)
plt.imshow(rgb_default, origin='lower')

We can also modify the scaling when computing the RGB image to allow for fainter features of the galaxies to show up better:

In [ ]:
rgb = make_lupton_rgb(i_data, r_data, g_data, Q=10, stretch=0.5, filename="ngc6976.jpeg")
plt.imshow(rgb, origin='lower')

Note: this image is now saved in this tutorial's main directory as `"ngc6976.jpeg"`. Go take a look!

As a final step - in the cell below, right some code that re-plots this image using the WCS in the header to assign the axes.

In [ ]:
#YOUR CODE HERE

***

#### Q - What types of astronomical objects do you see in this frame for NGC6976?

***

Ans.

That's it for today! Congrats on successfully completing Part 2 of our workshop's AstroPy FITS tutorial! 

***

# EXTRA

There is a third image associated with the first dataset we explored today, the Bubble Nebula (NGC 7635, Caldwell 11). You can find this V-filter image hosted here: https://www.dropbox.com/s/dzneit6oaevmm9j/ogg0m406-kb27-20201107-0172-e91.fits?dl=0

Load in this file using `wget` and make an RGB 3-color image of The Bubble Nebula where R is red, V is green, and B is blue!

